In [2]:
import numpy as np
from qequlacs.simulator import QulacsSimulator
from zquantum.core.estimation import calculate_exact_expectation_values
from zquantum.core.cost_function import AnsatzBasedCostFunction
from zquantum.core.graph import load_graph, generate_graph_from_specs
from zquantum.core.interfaces.optimizer import optimization_result
from zquantum.optimizers.scipy_optimizer import ScipyOptimizer
from zquantum.qaoa.ansatzes.farhi_ansatz import QAOAFarhiAnsatz
from zquantum.qaoa.problems.maxcut import get_maxcut_hamiltonian
from zquantum.qaoa.problems.stable_set import get_stable_set_hamiltonian
from qequlacs import QulacsSimulator
import time
import os

/home/abarthe/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/abarthe/.local/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


https://arxiv.org/pdf/1812.04170.pdf

In [4]:
def load_graph_from_file(graph_size, graph_id):
    return load_graph(
        os.path.join(
            "graphs", f"size_{graph_size}", f"graph_{graph_size}_{graph_id}.json"
        )
    )


def get_problem_hamiltonian(graph, problem="maxcut"):
    if problem == "maxcut":
        return get_maxcut_hamiltonian(graph)
    if problem == "stable_set":
        return get_stable_set_hamiltonian(graph)


def get_graph(size_of_graph, graph_id, graph_specs=None):
    if graph_specs is None:
        return load_graph_from_file(size_of_graph, graph_id)
    graph_specs["num_nodes"] = size_of_graph
    return generate_graph_from_specs(graph_specs)


def find_appropriate_params(cost_hamiltonian, ansatz, initial_params, mode):

    backend = QulacsSimulator()
    optimizer = ScipyOptimizer(method="L-BFGS-B")
    estimation_method = calculate_exact_expectation_values
    ####################
    # Modification suggestions
    ####################
    # from qeqiskit import QiskitSimulator()
    # backend = QiskitSimulator()

    # from zquantum.optimizers.basin_hopping import BasinHoppingOptimizer
    # optimizer = BasinHoppingOptimizer()
    # from zquantum.optimizers.cma_es_optimizer import CMAESOptimizer
    # optimizer = CMAESOptimizer(sigma_0=0.8)

    # from zquantum.qaoa.estimators import CvarEstimator
    # estimation_method = CvarEstimator(alpha=0.8)

    if mode == "low":
        cost_function = AnsatzBasedCostFunction(
            target_operator=-cost_hamiltonian,
            ansatz=ansatz,
            backend=backend,
            estimation_method=estimation_method,
        )
    else:
        cost_function = AnsatzBasedCostFunction(
            target_operator=cost_hamiltonian,
            ansatz=ansatz,
            backend=backend,
            estimation_method=estimation_method,
        )

    if mode in {"random", "evaluate"}:
        opt_results = optimization_result(
            opt_value=cost_function(initial_params),
            opt_params=initial_params,
        )
    else:
        opt_results = optimizer.minimize(cost_function, initial_params)

    if mode == "low":
        opt_results.opt_value = -opt_results.opt_value
    return opt_results


In [ ]:
####################
# 1. Simulation parameters
####################
size_of_graph = 10
size_of_big_graph = 20
number_of_graphs = 25
min_layers = 2
max_layers = 7
modes = ["high", "random", "low"]
problem = "maxcut"
# problem = "stable_set"
graph_specs = None
# graph_specs = {"type_graph": "regular", "degree": 3}
# graph_specs = {"type_graph": "regular", "degree": 4}
# graph_specs = {"type_graph": "erdos_renyi", "probability": 0.6}
# graph_specs = {"type_graph": "erdos_renyi", "probability": 0.8}

start_time = time.time()
####################
# 2. Layers loop
####################
for number_of_layers in range(min_layers, max_layers + 1):
    ####################
    # 3. Problem definition
    ####################
    print("Number of layers:", number_of_layers)
    graph_id = 0
    graph = get_graph(size_of_graph, graph_id, graph_specs)
    cost_hamiltonian = get_problem_hamiltonian(graph, problem)
    ansatz = QAOAFarhiAnsatz(
        number_of_layers=number_of_layers, cost_hamiltonian=cost_hamiltonian
    )

    ####################
    # 4. Finding parameters
    ####################
    selected_params = {}
    for mode in modes:
        number_of_params = ansatz.number_of_params
        initial_params = np.random.uniform(-np.pi, np.pi, number_of_params)
        opt_results = find_appropriate_params(
            cost_hamiltonian, ansatz, initial_params, mode
        )
        selected_params[mode] = opt_results.opt_params

    ####################
    # 5. Evaluating parameters
    ####################
    for evaluation_size in [size_of_graph, size_of_big_graph]:
        print("Graph size: ", evaluation_size)
        values = {mode: [] for mode in modes}
        for graph_id in range(number_of_graphs):
            ####################
            # 5.1 Problem definition
            ####################

            graph = load_graph_from_file(evaluation_size, graph_id)
            cost_hamiltonian = get_problem_hamiltonian(graph, problem)
            ansatz = QAOAFarhiAnsatz(
                number_of_layers=number_of_layers, cost_hamiltonian=cost_hamiltonian
            )
            ####################
            # 5.2 Execution
            ####################
            for mode in modes:
                params = selected_params[mode]
                opt_results = find_appropriate_params(
                    cost_hamiltonian, ansatz, params, "evaluate"
                )
                values[mode].append(opt_results.opt_value)

        ####################
        # 6. Data presentation
        ####################
        for mode in modes:
            print(mode, np.mean(values[mode]), np.std(values[mode]))

print("Total time:", time.time() - start_time)

Number of layers: 2


/home/abarthe/.local/lib/python3.8/site-packages/qiskit/optimization/applications/ising/max_cut.py:54: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  return WeightedPauliOperator(paulis=pauli_list), shift


Graph size:  10
high -10.851475983168141 0.12309735231713013
random -6.074470655648758 0.22187440531439387
low -3.1677372383301035 0.18152908088279113
Graph size:  20
high -21.749981264902434 0.14767730080422709
random -12.42708001988083 0.28282137527651124
low -6.862944691377638 0.2910215664758569
Number of layers: 3
Graph size:  10
high -11.613521154995768 0.22910022659599136
random -8.26125562915754 0.1405459970884415
low -1.8153125764498332 0.06821168806221772
Graph size:  20
high -23.040212132004502 0.24148427562424904
random -16.65662622343995 0.16931198637255584
low -4.96875422604623 0.3332410001520674
Number of layers: 4
Graph size:  10
high -11.401801337890497 0.19577994652181768
random -8.158413523120318 0.21956898422053586
low -1.77717293002701 0.05343405424672832
Graph size:  20
high -22.726833549257112 0.22140882635970843
random -16.400413823984046 0.20510879573491075
low -4.951142297056037 0.2845956895288959
Number of layers: 5
Graph size:  10
high -12.014668750133513 0.1